In [6]:
!pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 431 kB 4.6 MB/s 
     |████████████████████████████████| 4.9 MB 44.8 MB/s 
     |████████████████████████████████| 115 kB 62.3 MB/s 
     |████████████████████████████████| 163 kB 73.3 MB/s 
     |████████████████████████████████| 212 kB 72.6 MB/s 
     |████████████████████████████████| 127 kB 67.5 MB/s 
     |████████████████████████████████| 6.6 MB 44.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [7]:
!unzip data

Archive:  data.zip
   creating: data/dev/
  inflating: data/dev/EP3284738A1.txt  
  inflating: data/dev/EP3305797A1.txt  
  inflating: data/dev/EP3312182A1.txt  
  inflating: data/dev/EP3323817A1.txt  
  inflating: data/dev/EP3330263A1.txt  
  inflating: data/dev/EP3674305A1.txt  
  inflating: data/dev/EP3708563A1.txt  
  inflating: data/dev/US10654800B1.txt  
  inflating: data/dev/US10815179B1.txt  
  inflating: data/dev/US20180002276A1.txt  
  inflating: data/dev/US20180008574A1.txt  
  inflating: data/dev/US20180016269A1.txt  
  inflating: data/dev/US20180057448A1.txt  
  inflating: data/dev/US20180072767A1.txt  
  inflating: data/dev/US20180083209A1.txt  
  inflating: data/dev/US20180092937A1.txt  
  inflating: data/dev/US20180105741A1.txt  
  inflating: data/dev/US20200031833A1.txt  
  inflating: data/dev/US20200048291A1.txt  
  inflating: data/dev/US20200055807A1.txt  
  inflating: data/dev/US20200121691A1.txt  
  inflating: data/dev/US20200123187A1.txt  
  inflating: data/dev/US

In [8]:
import os
import os.path as osp
import nltk
import random
# nltk.download('stopwords')
# nltk.download('punkt')
# from nltk.corpus import stopwords
# english_stopwords = stopwords.words("english")
import numpy as np
import re
import seaborn as sns
sns.set_theme(style="whitegrid")
import matplotlib.pyplot as plt
import pandas as pd
import string
import torch
import datasets
from datasets import load_dataset
import pickle
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import multiprocessing

In [9]:
dev_path = "/content/data/dev"
train_path = "/content/data/train"

dev_label_path = "/content/dev_labels.pkl"
train_label_path = "/content/train_labels.pkl"

In [10]:
dev_files = [osp.join(dev_path, f)  for f in os.listdir(dev_path) if f[-3:]=='txt']
print(len(dev_files))
print(dev_files)

30
['/content/data/dev/US20200377536A1.txt', '/content/data/dev/US10654800B1.txt', '/content/data/dev/US20200164075A1.txt', '/content/data/dev/US20200308177A1.txt', '/content/data/dev/EP3305797A1.txt', '/content/data/dev/US20200031833A1.txt', '/content/data/dev/US20180002276A1.txt', '/content/data/dev/US20200123187A1.txt', '/content/data/dev/EP3674305A1.txt', '/content/data/dev/EP3284738A1.txt', '/content/data/dev/US20180016269A1.txt', '/content/data/dev/EP3323817A1.txt', '/content/data/dev/US20200181061A1.txt', '/content/data/dev/US20200048291A1.txt', '/content/data/dev/US20180072767A1.txt', '/content/data/dev/EP3708563A1.txt', '/content/data/dev/US20180105741A1.txt', '/content/data/dev/US20180092937A1.txt', '/content/data/dev/US20180008574A1.txt', '/content/data/dev/US20180057448A1.txt', '/content/data/dev/US20200165244A1.txt', '/content/data/dev/US20200270285A1.txt', '/content/data/dev/US20200148885A1.txt', '/content/data/dev/US20180083209A1.txt', '/content/data/dev/US20200121691A1.

We keep the dev set for Testing and take 20 documents out of the 120 documents in the train set for validation

In [11]:
train_files = [osp.join(train_path, f) for f in os.listdir(train_path) if f[-3:]=='txt']
print(len(train_files))
random.shuffle(train_files)

val_files = train_files[:20]
train_files = train_files[20:]

print(len(train_files))
print(len(val_files))

120
100
20


In [12]:
def create_dataset(files, labels_dict, output_file):

  train_data = {'para':[], 'label':[]}

  for f in files:
    with open(f, encoding="utf-8" ) as fp:
        
      lines = fp.readlines()
      train_data['para'].extend(lines)

      labels = labels_dict[f.split('/')[-1][:-4]] #".txt"
      train_data['label'].extend(labels)

  nl = []
  for l in train_data['label']:
    if(l==1):
      nl.append("1")
    else:
      nl.append("0")
    
  train_data['label'] = nl

  df = pd.DataFrame(train_data)
  print(df.head())
  df.to_csv(output_file, index = False)


In [13]:
create_dataset(dev_files, pickle.load(open(dev_label_path, "rb")), "dev_data.csv") # test set
create_dataset(train_files, pickle.load(open(train_label_path, "rb")), "train_data.csv") #train_set
create_dataset(val_files, pickle.load(open(train_label_path, "rb")), "val_data.csv") # validation set

                                                para label
0                                  TECHNICAL FIELD\n     0
1  The present invention relates to an optically ...     0
2                                   BACKGROUND ART\n     0
3  An optically active phosphine ligand having an...     0
4  In Patent Literature 2, a 2,3-bis(dialkylphosp...     0
                                                para label
0                      BACKGROUND OF THE INVENTION\n     0
1  This specification discloses, inter alia, drug...     0
2  Double helical DNA has two longitudinal spiral...     0
3  The width of the minor groove is approximately...     0
4  One type of MGB is a dimer of two polyaromatic...     0
                                                para label
0  The present invention relates to phosphorescen...     0
1  In OLEDs, the electroluminescence (EL) charact...     0
2  In other words, in case of organic light-emitt...     0
3                                     1. Substrate\n    

## Base BERT

In [14]:
#load csv files
dataset = load_dataset("csv", 
                       data_files={"train": "train_data.csv",
                                   "test": "val_data.csv"})
dataset

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-b4aafca8b243cb17/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['para', 'label'],
        num_rows: 44812
    })
    test: Dataset({
        features: ['para', 'label'],
        num_rows: 8751
    })
})

In [15]:
# model we will be using throughout
pretrained_model = "bert-base-uncased"

#load pre-trained tokenizer
tokenizer = AutoTokenizer.from_pretrained(pretrained_model, do_lower=True)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [16]:
s = "2-ethynylpyridine was replaced by 2-ethynyl-6-methylpyridine, 4-fluoroaniline was replaced by 3-aminopyridine, while the remaining raw materials, reagents and the preparation method were the same as those in Example 1 to give the product ZD080, yield 80%."
tokenized_output = tokenizer(s)
print("tokenizer output:\n", tokenized_output, "\n")
print("input_ids to tokens:\n", tokenizer.convert_ids_to_tokens(tokenized_output["input_ids"]))

tokenizer output:
 {'input_ids': [101, 1016, 1011, 3802, 10536, 4890, 14277, 12541, 28173, 2638, 2001, 2999, 2011, 1016, 1011, 3802, 10536, 4890, 2140, 1011, 1020, 1011, 25003, 7685, 14615, 3170, 1010, 1018, 1011, 19857, 14604, 7088, 4179, 2001, 2999, 2011, 1017, 1011, 13096, 7685, 14615, 3170, 1010, 2096, 1996, 3588, 6315, 4475, 1010, 2128, 4270, 7666, 1998, 1996, 7547, 4118, 2020, 1996, 2168, 2004, 2216, 1999, 2742, 1015, 2000, 2507, 1996, 4031, 1062, 2094, 2692, 17914, 1010, 10750, 3770, 1003, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [17]:
def preprocess(examples):
    return tokenizer(examples["para"], truncation=True, 
                     padding="max_length", max_length=128,
                     return_token_type_ids=False)

n_cpu = multiprocessing.cpu_count()

#preprocess
dataset = dataset.map(preprocess,# preprocessing function
                      num_proc=n_cpu,# number of workers to use
                      remove_columns=["para"] # columns to remove after preprocessing
                     )
dataset

#0:   0%|          | 0/22406 [00:00<?, ?ex/s]

#1:   0%|          | 0/22406 [00:00<?, ?ex/s]

#0:   0%|          | 0/4376 [00:00<?, ?ex/s]

#1:   0%|          | 0/4375 [00:00<?, ?ex/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 44812
    })
    test: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 8751
    })
})

In [20]:
model = AutoModelForSequenceClassification.from_pretrained(pretrained_model)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0e9f43ffc3afb401b28131d3443a63cb98e5deb8/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.22.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0e9f43ffc3afb401b28131d3443a63cb98e5deb8/pytorch_m

In [21]:
from transformers import TrainingArguments, Trainer
from sklearn.metrics import precision_recall_fscore_support

def compute_metrics(x):
    predictions, targets = x
    predictions = predictions.argmax(axis=1)
    scores = precision_recall_fscore_support(
        targets, predictions, average="weighted", zero_division=0.
        )
    
    return {
        "accuracy": (predictions == targets).sum() / len(predictions),
        "precision": scores[0],
        "recall": scores[1],
        "f-1": scores[2]
    }

batch_size = 64 # defined previously
seed = 8 # defined previously
lr = 5e-5 # learning rate
weight_decay = 1e-5
warmup_steps = 10 # scheduler #warmup steps
n_epochs = 5  # for the time sake. you would often want to train for longer epochs.

args = TrainingArguments(
    output_dir = "trained_model",
    evaluation_strategy="epoch",
    save_strategy="epoch", # could be epoch or step
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=n_epochs,
    weight_decay=weight_decay,
    warmup_steps=warmup_steps,
    load_best_model_at_end=True,
    metric_for_best_model="f-1", # metric defined in compute_metrics function
    seed=seed,
    dataloader_num_workers=n_cpu,
    disable_tqdm=False,
    save_total_limit=5, #limit number of checkpoints saved. Otherwise could take a lot of storage
)

trainer = Trainer(
    model,
    args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# start training
# metric shown below are for validation set
trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running training *****
  Num examples = 44812
  Num Epochs = 5
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 3505
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F-1
1,0.194800,0.323416,0.907553,0.906356,0.907553,0.906764
2,0.112400,0.454408,0.896126,0.893935,0.896126,0.894212
3,0.078500,0.428492,0.904354,0.902540,0.904354,0.902755
4,0.069600,0.496355,0.906982,0.905270,0.906982,0.905450
5,0.057700,0.538164,0.905268,0.903430,0.905268,0.903407


***** Running Evaluation *****
  Num examples = 8751
  Batch size = 64
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Saving model checkpoint to trained_model/checkpoint-701
Configuration saved in trained_model/checkpoint-701/config.json
Model weights saved in trained_model/checkpoint-701/pytorch_model.bin
tokenizer config file saved in trained_model/checkpoint-701/tokenizer_config.json
Special tokens file saved in trained_model/checkpoint-701/special_tokens_map.json
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster t

TrainOutput(global_step=3505, training_loss=0.09583820005966491, metrics={'train_runtime': 4858.5015, 'train_samples_per_second': 46.117, 'train_steps_per_second': 0.721, 'total_flos': 1.47381657659904e+16, 'train_loss': 0.09583820005966491, 'epoch': 5.0})

In [22]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 8751
  Batch size = 64
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 0.3234162926673889,
 'eval_accuracy': 0.9075534224660039,
 'eval_precision': 0.9063562549435307,
 'eval_recall': 0.9075534224660039,
 'eval_f-1': 0.906764082277502,
 'eval_runtime': 67.3784,
 'eval_samples_per_second': 129.878,
 'eval_steps_per_second': 2.033,
 'epoch': 5.0}

## Chemical domain pretrained BERT

In [ ]:
#load csv files
dataset = load_dataset("csv", 
                       data_files={"train": "train_data.csv",
                                   "test": "val_data.csv"})
dataset

In [ ]:

# model we will be using throughout
pretrained_model = "recobo/chemical-bert-uncased-pharmaceutical-chemical-classifier"

#load pre-trained tokenizer
tokenizer = AutoTokenizer.from_pretrained(pretrained_model, do_lower=True)


In [ ]:
s = "2-ethynylpyridine was replaced by 2-ethynyl-6-methylpyridine, 4-fluoroaniline was replaced by 3-aminopyridine, while the remaining raw materials, reagents and the preparation method were the same as those in Example 1 to give the product ZD080, yield 80%."
tokenized_output = tokenizer(s)
print("tokenizer output:\n", tokenized_output, "\n")
print("input_ids to tokens:\n", tokenizer.convert_ids_to_tokens(tokenized_output["input_ids"]))

tokenizer output:
 {'input_ids': [102, 170, 579, 2311, 819, 654, 11024, 5978, 241, 6703, 214, 170, 579, 2311, 819, 654, 579, 370, 579, 6090, 11024, 5978, 422, 286, 579, 6696, 8212, 4564, 1169, 241, 6703, 214, 239, 579, 4020, 11024, 5978, 422, 969, 111, 4234, 6908, 2518, 422, 10969, 137, 111, 4737, 551, 267, 111, 855, 188, 1052, 121, 1143, 158, 147, 3226, 111, 2378, 25174, 8037, 30131, 422, 2210, 2833, 1863, 205, 103], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]} 

input_ids to tokens:
 ['[CLS]', '2', '-', 'eth', '##yn', '##yl', '##pyr', '##idine', 'was', 'replaced', 'by', '2', '

In [ ]:
def preprocess(examples):
    return tokenizer(examples["para"], truncation=True, 
                     padding="max_length", max_length=128,
                     return_token_type_ids=False)

n_cpu = multiprocessing.cpu_count()

#preprocess
dataset = dataset.map(preprocess,# preprocessing function
                      num_proc=n_cpu,# number of workers to use
                      remove_columns=["para"] # columns to remove after preprocessing
                     )
dataset

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(pretrained_model)

In [ ]:
from transformers import TrainingArguments, Trainer
from sklearn.metrics import precision_recall_fscore_support

def compute_metrics(x):
    predictions, targets = x
    predictions = predictions.argmax(axis=1)
    scores = precision_recall_fscore_support(
        targets, predictions, average="weighted", zero_division=0.
        )
    
    return {
        "accuracy": (predictions == targets).sum() / len(predictions),
        "precision": scores[0],
        "recall": scores[1],
        "f-1": scores[2]
    }

batch_size = 64 # defined previously
seed = 8 # defined previously
lr = 5e-5 # learning rate
weight_decay = 1e-5
warmup_steps = 10 # scheduler #warmup steps
n_epochs = 10  # for the time sake. you would often want to train for longer epochs.

args = TrainingArguments(
    output_dir = "trained_model",
    evaluation_strategy="epoch",
    save_strategy="epoch", # could be epoch or step
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=n_epochs,
    weight_decay=weight_decay,
    warmup_steps=warmup_steps,
    load_best_model_at_end=True,
    metric_for_best_model="f-1", # metric defined in compute_metrics function
    seed=seed,
    dataloader_num_workers=n_cpu,
    disable_tqdm=False,
    save_total_limit=5, #limit number of checkpoints saved. Otherwise could take a lot of storage
)

trainer = Trainer(
    model,
    args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# start training
# metric shown below are for validation set
trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 53563
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 8370
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F-1
1,0.086900,0.383405,0.907389,0.909397,0.907389,0.908217
2,0.083700,0.421650,0.903659,0.904499,0.903659,0.904045
3,0.064900,0.509990,0.895812,0.893583,0.895812,0.894307
4,0.058300,0.525625,0.899930,0.898349,0.899930,0.898956
5,0.048500,0.571997,0.899231,0.897239,0.899231,0.897907


***** Running Evaluation *****
  Num examples = 12871
  Batch size = 32
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Saving model checkpoint to trained_model/checkpoint-1674
Configuration saved in trained_model/checkpoint-1674/config.json
Model weights saved in trained_model/checkpoint-1674/pytorch_model.bin
tokenizer config file saved in trained_model/checkpoint-1674/tokenizer_config.json
Special tokens file saved in trained_model/checkpoint-1674/special_tokens_map.json
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is fa

TrainOutput(global_step=8370, training_loss=0.07086233072953127, metrics={'train_runtime': 5955.838, 'train_samples_per_second': 44.967, 'train_steps_per_second': 1.405, 'total_flos': 1.76162718228096e+16, 'train_loss': 0.07086233072953127, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 12871
  Batch size = 32
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 0.3834046423435211,
 'eval_accuracy': 0.9073887032864579,
 'eval_precision': 0.9093965934687788,
 'eval_recall': 0.9073887032864579,
 'eval_f-1': 0.9082173984082131,
 'eval_runtime': 92.787,
 'eval_samples_per_second': 138.716,
 'eval_steps_per_second': 4.343,
 'epoch': 5.0}